# Navigation of the robot


In [2]:
import tdmclient.notebook
from Navigation_utils import *
from tdmclient import ClientAsync, aw
#await tdmclient.notebook.start()

client = ClientAsync()
node = await client.wait_for_node()

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
%%run_python

# Let's intitialize the variables

speed0 = 100       #initial speed
speed_gain = 2     # 
obstThrL = 10      # Low threshold for which the robot exits the avoidance obstacle
obstThrH = 20      # High threshold for which the robot enters the avoidance obstacle
obstSpeedGain = 5  # gain associated to obstacle avoidance 
close_obst = 0     # number of sensors that detect an obstacle

obstacle = 0          # 0 = normal obstacle, 1 = obstacle avoidance obstacle
path_reached = 0
prox = [0,0,0,0,0,0,0] # obstacle detection array
obj_right = 0      # 1 = object detected on the right, 0 = object detected on the left

button_center_last = 0
timer_period[0] = 10
'''
@onevent
def button_center():
    motor_right=0
    motor_left=0
'''
@onevent
def button_center():  # J'ai créé ca pour simuler le fait que j'ai réatteint ton chemin en appuyant sur le bouton, mais en vrai ca sera avec une condition quand j'aurai tes segments et la position
    global path_reached, button_center_last

    if button_center and not button_center_last: 
        if (path_reached == 0):
            path_reached = 1
        else:
            path_reached = 0
        print(path_reached)

    button_center_last = button_center

@onevent
def timer0():
    global speed0, speed_gain, obstThrL, obstThrH, obstSpeedGain, obstacle, prox, motor_left_target, motor_right_target, leds_top, prox_horizontal, path_reached, close_obst, obj_right

    prox = prox_horizontal
    if obstacle == 0:
        for i in range (6):
            if prox[i] > obstThrH:
                obstacle = 1
    else:
        close_obst = 0          # number of sensors that detect an obstacle
        for i in range (6):
            if prox[i] > obstThrL:
                close_obst = close_obst + 1
                if i>2:
                    obj_right = 1
                else:
                    obj_right = 0
        if close_obst == 0:
            if path_reached == 1:
                obstacle = 0

    actuation = [0,0]
    if obstacle == 0:
        #global_navigation()
        leds_top = [0,0,0]
        motor_left_target = speed0
        motor_right_target = speed0
    else: 
        #obstacle_surrounding()  
        leds_top = [30,30,30]
        w_l = [40,  20, -20, -20, -40,  30, -10]
        w_r = [-40, -20, -20,  20,  40, -10,  30]
        x = [0,0,0,0,0,0,0]
        sensor_translation = 100

        if close_obst == 0:
            if (obj_right == 1):
                actuation[0] = 50
                actuation[1] = -25
            else:
                actuation[0] = -25
                actuation[1] = 50
        else:
            for i in range (6):
                x[i] = prox[i] // sensor_translation
                actuation[0] = actuation[0] + x[i] * w_l[i]
                actuation[1] = actuation[1] + x[i] * w_r[i]

        motor_left_target = speed0 + actuation[0]
        motor_right_target = speed0 + actuation[1]

In [ ]:
await tdmclient.notebook.stop()

In [ ]:
def collision_avoidance(nodes_slopes, pose_est, error_est, prox, old_obstacle):
    speed_turn_left = [-25,50]       #Actuation for turning left
    speed_turn_right = [50,-25]      #Actuation for turning right
    actuation = [0,0]
    #obstThrL = 10      # Low threshold for which the robot exits the avoidance obstacle
    obstThrH = 20      # High threshold for which the robot enters the avoidance obstacle
    obstacle = 0         # 0 = no obstacle, 1 = obstacle avoidance 
    close_obst = 0      # number of sensors that detect an obstacle

    path_reached = 0    # 0 = not reached, 1 = reached
    obj_right = 0      # 1 = object detected on the right, 0 = object detected on the left
    
    await node.wait_for_variables()
    prox = node.get_variable("prox.horizontal")  # Lire les valeurs des capteurs de proximité

    #Let's check if we reached a segment of the global path
    for i in range (0, nodes_slopes.shape[0]-1):
        alpha = nodes_slopes[i,2]
        beta = nodes_slopes[i,3]
        x_path = [nodes_slopes[i,0], nodes_slopes[i,1]]
        y_path = [nodes_slopes[i+1,0], nodes_slopes[i+1,1]]
        x_path.sort()
        y_path.sort()
        if ((x_path[0] - error_est[0] <= pose_est[0] <= x_path[1] + error_est[0]) 
            and (y_path[1] - error_est[1] <= pose_est[1] <= y_path[1] + error_est[1])):
            pose_y_min = pose_est[1] - error_est[1]
            pose_y_max = pose_est[1] + error_est[1]

            fx_min = alpha * (pose_est[0] - error_est[0]) + beta
            fx_max = alpha * (pose_est[0] + error_est[0]) + beta

            if alpha < 0:                   # If alpha is negative, we have to inverse the boundaries
                fx_min, fx_max = fx_max, fx_min

            if (pose_y_min <= fx_max) and (fx_min <= pose_y_max):
                path_reached = 1
            else:
                path_reached = 0
            
    if old_obstacle == 0:
        for i in range (6):
            if prox[i] > obstThrH:
                obstacle = 1
    else:          
        for i in range (6):     # find if the obstacle is on the left or on the right
            if prox[i] > obstThrH:
                close_obst = close_obst + 1
                if i>2:
                    obj_right = 1
                    obstacle = 1
                else:
                    obj_right = 0
                    obstacle = 1
        if close_obst == 0:
            if path_reached == 1:
                obstacle = 0
                return actuation, obstacle
    
    # Now, let's actuate the robot in order to surround the obstacle
    #leds_top = [30,30,30]
    w_l = [40,  20, -20, -20, -40,  30, -10]
    w_r = [-40, -20, -20,  20,  40, -10,  30]
    x = [0,0,0,0,0,0,0]
    sensor_translation = 100

    if (close_obst == 0):
        if (obj_right == 1):
            actuation[0] = speed_turn_right[0]
            actuation[1] = speed_turn_right[1]
        else:
            actuation[0] = speed_turn_left[0]
            actuation[1] = speed_turn_left[1]
    else:
        for i in range (6):
            x[i] = prox[i] // sensor_translation
            actuation[0] = actuation[0] + x[i] * w_l[i]
            actuation[1] = actuation[1] + x[i] * w_r[i]

        return actuation, obstacle
